In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Howard\\PycharmProjects\\mlyt\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\Howard\\PycharmProjects\\mlyt'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/hlauhh/mlyt.mlflow",

        )

        return model_evaluation_config


In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        dagshub.init(repo_owner='hlauhh', repo_name='mlyt', mlflow=True)

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2



    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")




In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-28 23:01:30,526: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-28 23:01:30,527: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-28 23:01:30,528: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-28 23:01:30,529: INFO: common: created directory at: artifacts]
[2024-12-28 23:01:30,530: INFO: common: created directory at: artifacts/model_evaluation]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=cccd9601-27fa-43eb-8d8e-c0e73b4a5c7a&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ce7cb409128fcdd5ba527a92638cfe84cf436ce85684bf90008b0cdaa345bff0




C:\Users\Howard\PycharmProjects\mlyt\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2024-12-28 23:01:38,485: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


...

Initialized MLflow to track repo "hlauhh/mlyt"

[2024-12-28 23:01:41,029: INFO: helpers: Initialized MLflow to track repo "hlauhh/mlyt"]


Repository hlauhh/mlyt initialized!

[2024-12-28 23:01:41,031: INFO: helpers: Repository hlauhh/mlyt initialized!]
[2024-12-28 23:01:41,795: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


C:\Users\Howard\PycharmProjects\mlyt\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2024/12/28 23:01:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
